In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

## Загрузка данных

In [2]:
data = pd.read_csv('./data/amine_treatment-final.csv', on_bad_lines='skip')
data

,"feed_gas temperature, C","feed_gas pressure, kPa","feed_gas mass flow, kg/h",feed_gas molecular weight,"feed_gas Mass density, kg/m3","feed_gas actual liquid flow, m3/h","feed_gas actual vapour flow, m3/h",feed_gas CO2 mol frac,feed_gas Methane mol frac,feed_gas Ethane mol frac,...,degased_amine Propane mol frac,degased_amine i-Butane mol frac,degased_amine n-Butane mol frac,degased_amine i-Pentane mol frac,degased_amine n-Pentane mol frac,degased_amine H2S mol frac,degased_amine H2O mol frac,degased_amine MDEAmine mol frac,degased_amine H2S ppm,degased_amine CO2 ppm
0,52.0,6701.325,259647.0,24.668523,80.705482,0.000000,3217.216383,0.031875,0.633122,0.192729,...,0.000004,1.979803e-06,6.785930e-06,3.039502e-08,1.629357e-07,0.014838,0.910916,0.062270,22521.880092,23245.607009
1,40.0,6701.325,63853.0,23.411939,79.292250,0.000000,805.286774,0.039520,0.681947,0.179992,...,0.000006,1.099164e-07,7.886635e-07,2.425456e-06,2.679624e-08,0.010470,0.896612,0.077190,15891.219056,30468.484150
2,46.0,6701.325,15928.0,27.803853,108.358884,0.000000,146.993024,0.027859,0.577870,0.163095,...,0.000018,5.179434e-05,2.863001e-05,4.721545e-06,1.359401e-06,0.001284,0.860268,0.132676,1949.372531,10648.306282
3,36.0,6701.325,150276.0,22.053344,74.092459,0.000000,2028.222609,0.005420,0.856717,0.000249,...,0.000039,8.734358e-06,3.706189e-05,5.426945e-05,3.949909e-06,0.001233,0.809722,0.184359,1871.528366,8438.736973
4,12.0,6701.325,52691.0,24.334510,114.182302,0.002870,461.460940,0.027297,0.694634,0.091063,...,0.000100,7.719691e-05,2.661312e-05,1.351350e-05,6.619552e-06,0.003300,0.788340,0.200715,5008.207346,14011.150105
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
512795,18.0,6701.325,278813.0,27.267819,147.437736,0.239690,1890.816202,0.036621,0.666807,0.042159,...,0.000053,1.226987e-04,3.336251e-05,4.145371e-06,8.510407e-06,0.002101,0.796328,0.187149,3188.443718,27411.897443
512796,33.0,6701.325,72612.0,26.629816,112.530303,0.000000,645.266193,0.025383,0.605142,0.150361,...,0.000049,8.271254e-05,1.199104e-05,1.810930e-06,4.749955e-06,0.003784,0.826966,0.162727,5742.847844,11941.343733
512797,17.0,6701.325,272718.0,26.290335,135.390018,0.133398,2014.180541,0.039356,0.606850,0.167318,...,0.000037,2.874297e-05,3.343060e-06,1.045919e-05,2.187381e-06,0.045335,0.725301,0.179546,68812.636471,97071.672175
512798,59.0,6701.325,75970.0,24.231673,73.727565,0.000000,1030.415146,0.037951,0.732049,0.086286,...,0.000013,1.782460e-05,8.286743e-06,5.417825e-06,2.393693e-07,0.002551,0.877567,0.106280,3871.441495,26287.366187


## Модель для определения температуры продуктов

In [3]:
mini_data = data.sample(frac=0.25)
len(mini_data)

128200

In [5]:
list(mini_data.columns)

['feed_gas temperature, C',
 'feed_gas pressure, kPa',
 'feed_gas mass flow, kg/h',
 'feed_gas molecular weight',
 'feed_gas Mass density, kg/m3',
 'feed_gas actual liquid flow, m3/h',
 'feed_gas actual vapour flow, m3/h',
 'feed_gas CO2 mol frac',
 'feed_gas Methane mol frac',
 'feed_gas Ethane mol frac',
 'feed_gas Propane mol frac',
 'feed_gas i-Butane mol frac',
 'feed_gas n-Butane mol frac',
 'feed_gas i-Pentane mol frac',
 'feed_gas n-Pentane mol frac',
 'feed_gas H2S mol frac',
 'feed_gas H2O mol frac',
 'feed_gas MDEAmine mol frac',
 'feed_gas H2S ppm',
 'feed_gas CO2 ppm',
 'Absorber number of stages',
 'lean_amine temperature, C',
 'lean_amine pressure, kPa',
 'lean_amine mass flow, kg/h',
 'lean_amine molecular weight',
 'lean_amine Mass density, kg/m3',
 'lean_amine actual liquid flow, m3/h',
 'lean_amine actual vapour flow, m3/h',
 'lean_amine CO2 mol frac',
 'lean_amine Methane mol frac',
 'lean_amine Ethane mol frac',
 'lean_amine Propane mol frac',
 'lean_amine i-Butane

In [6]:
sulphur = data[['feed_gas temperature, C', 'feed_gas mass flow, kg/h', 'feed_gas CO2 mol frac',
                'feed_gas Methane mol frac', 'feed_gas Ethane mol frac', 'feed_gas Propane mol frac', 
                'feed_gas i-Butane mol frac', 'feed_gas n-Butane mol frac', 'feed_gas i-Pentane mol frac',
                'feed_gas n-Pentane mol frac', 'feed_gas H2S mol frac', 'feed_gas H2O mol frac',
                'feed_gas MDEAmine mol frac',
                'lean_amine temperature, C', 'lean_amine mass flow, kg/h', 'lean_amine CO2 mol frac',
                'lean_amine Methane mol frac', 'lean_amine Ethane mol frac', 'lean_amine Propane mol frac',
                'lean_amine i-Butane mol frac', 'lean_amine n-Butane mol frac', 'lean_amine i-Pentane mol frac',
                'lean_amine n-Pentane mol frac', 'lean_amine H2S mol frac', 'lean_amine H2O mol frac',
                'lean_amine MDEAmine mol frac', 'sweet_gas temperature, C', 'rich_amine temperature, C'
               ]]

X = sulphur[[
                'feed_gas temperature, C', 'feed_gas mass flow, kg/h', 'feed_gas CO2 mol frac',
                'feed_gas Methane mol frac', 'feed_gas Ethane mol frac', 'feed_gas Propane mol frac', 
                'feed_gas i-Butane mol frac', 'feed_gas n-Butane mol frac', 'feed_gas i-Pentane mol frac',
                'feed_gas n-Pentane mol frac', 'feed_gas H2S mol frac', 'feed_gas H2O mol frac',
                'feed_gas MDEAmine mol frac',
                'lean_amine temperature, C', 'lean_amine mass flow, kg/h', 'lean_amine CO2 mol frac',
                'lean_amine Methane mol frac', 'lean_amine Ethane mol frac', 'lean_amine Propane mol frac',
                'lean_amine i-Butane mol frac', 'lean_amine n-Butane mol frac', 'lean_amine i-Pentane mol frac',
                'lean_amine n-Pentane mol frac', 'lean_amine H2S mol frac', 'lean_amine H2O mol frac',
                'lean_amine MDEAmine mol frac'
]]

y = sulphur[['sweet_gas temperature, C', 'rich_amine temperature, C']]

# Собираем трансформер столбцов
ct = make_column_transformer(
    (MinMaxScaler(), ['feed_gas temperature, C', 'feed_gas mass flow, kg/h', 'feed_gas CO2 mol frac',
                'feed_gas Methane mol frac', 'feed_gas Ethane mol frac', 'feed_gas Propane mol frac', 
                'feed_gas i-Butane mol frac', 'feed_gas n-Butane mol frac', 'feed_gas i-Pentane mol frac',
                'feed_gas n-Pentane mol frac', 'feed_gas H2S mol frac', 'feed_gas H2O mol frac',
                'feed_gas MDEAmine mol frac',
                'lean_amine temperature, C', 'lean_amine mass flow, kg/h', 'lean_amine CO2 mol frac',
                'lean_amine Methane mol frac', 'lean_amine Ethane mol frac', 'lean_amine Propane mol frac',
                'lean_amine i-Butane mol frac', 'lean_amine n-Butane mol frac', 'lean_amine i-Pentane mol frac',
                'lean_amine n-Pentane mol frac', 'lean_amine H2S mol frac', 'lean_amine H2O mol frac',
                'lean_amine MDEAmine mol frac'])
)

# Создаем обучающую и тестовые выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Загружаем тренировочную выборку
ct.fit(X_train)

# Нормализуем данные
X_train_normal = ct.transform(X_train)
X_test_normal = ct.transform(X_test)

X_train_normal.shape, y_train.shape

((410240, 26), (410240, 2))

In [86]:
# Set random set
tf.random.set_seed(42)

# 1. Create a model
prod_temp_model = tf.keras.Sequential([
    tf.keras.layers.Dense(26, activation='relu'),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(2)
])

# 2. Compile the model
prod_temp_model.compile(
    loss='mae',
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    metrics=['mae']
)

# 3. Fit the model
prod_temp_model.fit(X_train_normal, y_train, epochs=20)

Epoch 1/20
12820/12820 [==============================] - 31s 2ms/step - loss: 2.8807 - mae: 2.8807
Epoch 2/20
12820/12820 [==============================] - 31s 2ms/step - loss: 1.7489 - mae: 1.7489
Epoch 3/20
12820/12820 [==============================] - 31s 2ms/step - loss: 1.5962 - mae: 1.5962
Epoch 4/20
12820/12820 [==============================] - 32s 2ms/step - loss: 1.4959 - mae: 1.4959
Epoch 5/20
12820/12820 [==============================] - 31s 2ms/step - loss: 1.4451 - mae: 1.4451
Epoch 6/20
12820/12820 [==============================] - 32s 2ms/step - loss: 1.4027 - mae: 1.4027
Epoch 7/20
12820/12820 [==============================] - 31s 2ms/step - loss: 1.3708 - mae: 1.3708
Epoch 8/20
12820/12820 [==============================] - 32s 2ms/step - loss: 1.3418 - mae: 1.3418
Epoch 9/20
12820/12820 [==============================] - 31s 2ms/step - loss: 1.3195 - mae: 1.3195
Epoch 10/20
12820/12820 [==============================] - 32s 2ms/step - loss: 1.3019 - mae: 1.3019

In [87]:
y_pred = prod_temp_model.predict(X_test_normal)

3205/3205 [==============================] - 4s 1ms/step


In [107]:
y_pred[101]

array([57.381298, 55.735657], dtype=float32)

In [108]:
y_test.iloc[101]

sweet_gas temperature, C     57.480083
rich_amine temperature, C    55.968702
Name: 237204, dtype: float64

In [110]:
100 - ((55.735657 * 100) / 55.968702)

0.4163845000371822

In [111]:
prod_temp_model.save('./models/amine_treatment/amine_treatment_prod_temp_model.h5')

In [12]:
sulphur_data = sulphur.sample(frac=0.01)
sulphur_data

,"feed_gas temperature, C","feed_gas mass flow, kg/h",feed_gas CO2 mol frac,feed_gas Methane mol frac,feed_gas Ethane mol frac,feed_gas Propane mol frac,feed_gas i-Butane mol frac,feed_gas n-Butane mol frac,feed_gas i-Pentane mol frac,feed_gas n-Pentane mol frac,...,lean_amine Propane mol frac,lean_amine i-Butane mol frac,lean_amine n-Butane mol frac,lean_amine i-Pentane mol frac,lean_amine n-Pentane mol frac,lean_amine H2S mol frac,lean_amine H2O mol frac,lean_amine MDEAmine mol frac,"sweet_gas temperature, C","rich_amine temperature, C"
229777,59.0,295333.0,0.037486,0.709195,0.027945,0.049124,0.096944,0.016101,0.022952,0.004392,...,0.0,0.0,0.0,0.0,0.0,0.000211,0.977453,0.019753,63.534447,65.537233
230248,22.0,51765.0,0.036442,0.649914,0.060758,0.072099,0.052338,0.046696,0.041313,0.003336,...,0.0,0.0,0.0,0.0,0.0,0.000344,0.810059,0.184826,25.056876,30.451538
142441,53.0,94584.0,0.026240,0.633435,0.186715,0.090658,0.019003,0.011180,0.025023,0.002668,...,0.0,0.0,0.0,0.0,0.0,0.000291,0.890271,0.108304,26.411256,36.111568
236140,39.0,135235.0,0.028697,0.800068,0.024626,0.082167,0.009194,0.023052,0.017758,0.006773,...,0.0,0.0,0.0,0.0,0.0,0.000181,0.839054,0.157503,35.072823,45.195079
328006,50.0,27226.0,0.024775,0.728327,0.096035,0.039802,0.018610,0.062545,0.017170,0.009563,...,0.0,0.0,0.0,0.0,0.0,0.000238,0.899695,0.097257,40.644903,46.896273
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127774,57.0,106247.0,0.007134,0.564712,0.150123,0.092254,0.052006,0.076666,0.035214,0.003127,...,0.0,0.0,0.0,0.0,0.0,0.000139,0.860390,0.136670,33.316939,47.659560
340722,12.0,139188.0,0.014490,0.654710,0.126441,0.074161,0.058075,0.011644,0.014428,0.007704,...,0.0,0.0,0.0,0.0,0.0,0.000116,0.858768,0.137278,23.306435,25.228895
460614,20.0,278135.0,0.020406,0.661170,0.083639,0.022305,0.068197,0.078623,0.049696,0.000717,...,0.0,0.0,0.0,0.0,0.0,0.000386,0.845761,0.152531,21.611649,26.587039
437188,43.0,94367.0,0.028831,0.658443,0.177795,0.036555,0.024317,0.051922,0.007627,0.000021,...,0.0,0.0,0.0,0.0,0.0,0.000317,0.980460,0.017175,32.290935,35.745604


In [13]:
sulphur_data.to_csv('sulphur_data.csv')

## Модель для определения массовых расходов продуктовых потоков

In [92]:
mini_data = data.sample(frac=0.85)
len(mini_data)

435880

In [93]:
list(mini_data.columns)

['feed_gas temperature, C',
 'feed_gas pressure, kPa',
 'feed_gas mass flow, kg/h',
 'feed_gas molecular weight',
 'feed_gas Mass density, kg/m3',
 'feed_gas actual liquid flow, m3/h',
 'feed_gas actual vapour flow, m3/h',
 'feed_gas CO2 mol frac',
 'feed_gas Methane mol frac',
 'feed_gas Ethane mol frac',
 'feed_gas Propane mol frac',
 'feed_gas i-Butane mol frac',
 'feed_gas n-Butane mol frac',
 'feed_gas i-Pentane mol frac',
 'feed_gas n-Pentane mol frac',
 'feed_gas H2S mol frac',
 'feed_gas H2O mol frac',
 'feed_gas MDEAmine mol frac',
 'feed_gas H2S ppm',
 'feed_gas CO2 ppm',
 'Absorber number of stages',
 'lean_amine temperature, C',
 'lean_amine pressure, kPa',
 'lean_amine mass flow, kg/h',
 'lean_amine molecular weight',
 'lean_amine Mass density, kg/m3',
 'lean_amine actual liquid flow, m3/h',
 'lean_amine actual vapour flow, m3/h',
 'lean_amine CO2 mol frac',
 'lean_amine Methane mol frac',
 'lean_amine Ethane mol frac',
 'lean_amine Propane mol frac',
 'lean_amine i-Butane

In [122]:
prod_temp = data[[
    'feed_gas temperature, C', 'feed_gas mass flow, kg/h', 'feed_gas CO2 mol frac', 'feed_gas Methane mol frac',
    'feed_gas Ethane mol frac', 'feed_gas Propane mol frac', 'feed_gas i-Butane mol frac', 'feed_gas n-Butane mol frac',
    'feed_gas i-Pentane mol frac', 'feed_gas n-Pentane mol frac', 'feed_gas H2S mol frac', 'feed_gas H2O mol frac',
    'feed_gas MDEAmine mol frac', 'lean_amine temperature, C', 'lean_amine mass flow, kg/h', 'lean_amine CO2 mol frac',
    'lean_amine Methane mol frac', 'lean_amine Ethane mol frac', 'lean_amine Propane mol frac',
    'lean_amine i-Butane mol frac', 'lean_amine n-Butane mol frac', 'lean_amine i-Pentane mol frac',
    'lean_amine n-Pentane mol frac', 'lean_amine H2S mol frac', 'lean_amine H2O mol frac', 'lean_amine MDEAmine mol frac',
     'rich_amine mass flow, kg/h', 'sweet_gas mass flow, kg/h'
               ]]

X = prod_temp[[
    'feed_gas temperature, C', 'feed_gas mass flow, kg/h', 'feed_gas CO2 mol frac', 'feed_gas Methane mol frac',
    'feed_gas Ethane mol frac', 'feed_gas Propane mol frac', 'feed_gas i-Butane mol frac', 'feed_gas n-Butane mol frac',
    'feed_gas i-Pentane mol frac', 'feed_gas n-Pentane mol frac', 'feed_gas H2S mol frac', 'feed_gas H2O mol frac',
    'feed_gas MDEAmine mol frac', 'lean_amine temperature, C', 'lean_amine mass flow, kg/h', 'lean_amine CO2 mol frac',
    'lean_amine Methane mol frac', 'lean_amine Ethane mol frac', 'lean_amine Propane mol frac',
    'lean_amine i-Butane mol frac', 'lean_amine n-Butane mol frac', 'lean_amine i-Pentane mol frac',
    'lean_amine n-Pentane mol frac', 'lean_amine H2S mol frac', 'lean_amine H2O mol frac', 'lean_amine MDEAmine mol frac',
]]

y = prod_temp[['rich_amine mass flow, kg/h', 'sweet_gas mass flow, kg/h']]

# Собираем трансформер столбцов
ct = make_column_transformer(
    (MinMaxScaler(), [
    'feed_gas temperature, C', 'feed_gas mass flow, kg/h', 'feed_gas CO2 mol frac', 'feed_gas Methane mol frac',
    'feed_gas Ethane mol frac', 'feed_gas Propane mol frac', 'feed_gas i-Butane mol frac', 'feed_gas n-Butane mol frac',
    'feed_gas i-Pentane mol frac', 'feed_gas n-Pentane mol frac', 'feed_gas H2S mol frac', 'feed_gas H2O mol frac',
    'feed_gas MDEAmine mol frac', 'lean_amine temperature, C', 'lean_amine mass flow, kg/h', 'lean_amine CO2 mol frac',
    'lean_amine Methane mol frac', 'lean_amine Ethane mol frac', 'lean_amine Propane mol frac',
    'lean_amine i-Butane mol frac', 'lean_amine n-Butane mol frac', 'lean_amine i-Pentane mol frac',
    'lean_amine n-Pentane mol frac', 'lean_amine H2S mol frac', 'lean_amine H2O mol frac', 'lean_amine MDEAmine mol frac',
    ])
)

# Создаем обучающую и тестовые выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Загружаем тренировочную выборку
ct.fit(X_train)

# Нормализуем данные
X_train_normal = ct.transform(X_train)
X_test_normal = ct.transform(X_test)

X_train_normal.shape, y_train.shape

((410240, 26), (410240, 2))

In [123]:
# Set random set
tf.random.set_seed(42)

# 1. Create a model
prod_mass_flow_model = tf.keras.Sequential([
    tf.keras.layers.Dense(26, activation='relu'),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(2)
])

# 2. Compile the model
prod_mass_flow_model.compile(
    loss='mae',
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    metrics=['mae']
)

# 3. Fit the model
prod_mass_flow_model.fit(X_train_normal, y_train, epochs=10)

Epoch 1/10
12820/12820 [==============================] - 28s 2ms/step - loss: 50311.9336 - mae: 50311.9336
Epoch 2/10
12820/12820 [==============================] - 24s 2ms/step - loss: 43860.3086 - mae: 43860.3086
Epoch 3/10
12820/12820 [==============================] - 28s 2ms/step - loss: 43816.8516 - mae: 43816.8516
Epoch 4/10
12820/12820 [==============================] - 33s 3ms/step - loss: 30582.1582 - mae: 30582.1582
Epoch 5/10
12820/12820 [==============================] - 33s 3ms/step - loss: 1560.9634 - mae: 1560.9634
Epoch 6/10
12820/12820 [==============================] - 28s 2ms/step - loss: 1439.8798 - mae: 1439.8798
Epoch 7/10
12820/12820 [==============================] - 28s 2ms/step - loss: 1413.0997 - mae: 1413.0997
Epoch 8/10
12820/12820 [==============================] - 28s 2ms/step - loss: 1395.4691 - mae: 1395.4691
Epoch 9/10
12820/12820 [==============================] - 31s 2ms/step - loss: 1379.0150 - mae: 1379.0150
Epoch 10/10
12820/12820 [=============

In [124]:
y_pred = prod_mass_flow_model.predict(X_test_normal)

3205/3205 [==============================] - 6s 2ms/step


In [125]:
y_pred

array([[509073.38 , 194866.1  ],
       [188156.56 ,  55655.934],
       [513431.72 , 185600.34 ],
       ...,
       [ 50163.203, 192702.7  ],
       [290650.7  ,  36081.438],
       [702582.5  , 249903.23 ]], dtype=float32)

In [126]:
y_test

,"rich_amine mass flow, kg/h","sweet_gas mass flow, kg/h"
337566,507815.414547,196179.585453
201781,187562.842578,56036.157422
215224,515275.821637,183408.178363
8834,283970.774396,148700.225604
394112,254584.050699,38068.949301
...,...,...
396710,176328.112169,255169.887831
447239,704293.420295,126707.579705
217230,49227.100639,193539.899361
397167,288826.341559,37772.658441


In [121]:
lean_amine_mass_flow_model.save('./models/amine_treatment/amine_treatment_lean_amine_mass_flow.h5')

## Модель для определения молярного расхода H2S в продуктовых потоках

In [159]:
data['rich_amine molar flow, kgmol/h'] = data['rich_amine mass flow, kg/h'] / data['rich_amine molecular weight']
data['rich_amine H2S molar flow, kgmol/h'] = data['rich_amine molar flow, kgmol/h'] * data['rich_amine H2S mol frac']
data['feed_gas molar flow, kgmol/h'] = data['feed_gas mass flow, kg/h'] / data['feed_gas molecular weight']
data['feed_gas H2S molar flow, kgmol/h'] = data['feed_gas molar flow, kgmol/h'] * data['feed_gas H2S mol frac']

data['lean_amine molar flow, kgmol/h'] = data['lean_amine mass flow, kg/h'] / data['lean_amine molecular weight']
data['lean_amine H2S molar flow, kgmol/h'] = data['lean_amine molar flow, kgmol/h'] * data['lean_amine H2S mol frac']

In [160]:
data['feed_gas H2S molar flow, kgmol/h'], data['lean_amine H2S molar flow, kgmol/h'], data['rich_amine H2S molar flow, kgmol/h']

(0         214.172325
 1          61.998302
 2          17.128909
 3           4.649367
 4          40.722867
              ...    
 512795     31.327454
 512796     39.714945
 512797     52.520044
 512798     13.222511
 512799     35.568963
 Name: feed_gas H2S molar flow, kgmol/h, Length: 512800, dtype: float64,
 0         4.209285
 1         1.561811
 2         1.832692
 3         1.426863
 4         1.755754
             ...   
 512795    6.298323
 512796    4.568171
 512797    0.110803
 512798    2.210424
 512799    6.144748
 Name: lean_amine H2S molar flow, kgmol/h, Length: 512800, dtype: float64,
 0         218.199124
 1          63.550086
 2          18.958503
 3           6.009298
 4          42.458704
              ...    
 512795     37.481195
 512796     44.264273
 512797     52.418111
 512798     15.402125
 512799     41.705379
 Name: rich_amine H2S molar flow, kgmol/h, Length: 512800, dtype: float64)

In [161]:
mini_data = data.sample(frac=0.85)
len(mini_data)

435880

In [162]:
list(mini_data.columns)

['feed_gas temperature, C',
 'feed_gas pressure, kPa',
 'feed_gas mass flow, kg/h',
 'feed_gas molecular weight',
 'feed_gas Mass density, kg/m3',
 'feed_gas actual liquid flow, m3/h',
 'feed_gas actual vapour flow, m3/h',
 'feed_gas CO2 mol frac',
 'feed_gas Methane mol frac',
 'feed_gas Ethane mol frac',
 'feed_gas Propane mol frac',
 'feed_gas i-Butane mol frac',
 'feed_gas n-Butane mol frac',
 'feed_gas i-Pentane mol frac',
 'feed_gas n-Pentane mol frac',
 'feed_gas H2S mol frac',
 'feed_gas H2O mol frac',
 'feed_gas MDEAmine mol frac',
 'feed_gas H2S ppm',
 'feed_gas CO2 ppm',
 'Absorber number of stages',
 'lean_amine temperature, C',
 'lean_amine pressure, kPa',
 'lean_amine mass flow, kg/h',
 'lean_amine molecular weight',
 'lean_amine Mass density, kg/m3',
 'lean_amine actual liquid flow, m3/h',
 'lean_amine actual vapour flow, m3/h',
 'lean_amine CO2 mol frac',
 'lean_amine Methane mol frac',
 'lean_amine Ethane mol frac',
 'lean_amine Propane mol frac',
 'lean_amine i-Butane

In [165]:
sulphur = mini_data[[
    'feed_gas temperature, C', 'feed_gas mass flow, kg/h', 'feed_gas CO2 mol frac', 'feed_gas Methane mol frac',
    'feed_gas Ethane mol frac', 'feed_gas Propane mol frac', 'feed_gas i-Butane mol frac', 'feed_gas n-Butane mol frac',
    'feed_gas i-Pentane mol frac', 'feed_gas n-Pentane mol frac', 'feed_gas H2S mol frac', 'feed_gas H2O mol frac',
    'feed_gas MDEAmine mol frac', 'feed_gas molar flow, kgmol/h', 'feed_gas H2S molar flow, kgmol/h',
    'lean_amine temperature, C', 'lean_amine mass flow, kg/h', 'lean_amine CO2 mol frac',
    'lean_amine Methane mol frac', 'lean_amine Ethane mol frac', 'lean_amine Propane mol frac',
    'lean_amine i-Butane mol frac', 'lean_amine n-Butane mol frac', 'lean_amine i-Pentane mol frac',
    'lean_amine n-Pentane mol frac', 'lean_amine H2S mol frac', 'lean_amine H2O mol frac', 'lean_amine MDEAmine mol frac',
    'lean_amine molar flow, kgmol/h', 'lean_amine H2S molar flow, kgmol/h',
    'rich_amine temperature, C', 'rich_amine mass flow, kg/h', 'rich_amine molecular weight', 'rich_amine H2S mol frac',
    'rich_amine molar flow, kgmol/h', 'rich_amine H2S molar flow, kgmol/h'
]]

In [166]:
X = sulphur[[
    'feed_gas temperature, C', 'feed_gas mass flow, kg/h', 'feed_gas CO2 mol frac', 'feed_gas Methane mol frac',
    'feed_gas Ethane mol frac', 'feed_gas Propane mol frac', 'feed_gas i-Butane mol frac', 'feed_gas n-Butane mol frac',
    'feed_gas i-Pentane mol frac', 'feed_gas n-Pentane mol frac', 'feed_gas H2S mol frac', 'feed_gas H2O mol frac',
    'feed_gas MDEAmine mol frac', 'feed_gas molar flow, kgmol/h', 'feed_gas H2S molar flow, kgmol/h',
    'lean_amine temperature, C', 'lean_amine mass flow, kg/h', 'lean_amine CO2 mol frac',
    'lean_amine Methane mol frac', 'lean_amine Ethane mol frac', 'lean_amine Propane mol frac',
    'lean_amine i-Butane mol frac', 'lean_amine n-Butane mol frac', 'lean_amine i-Pentane mol frac',
    'lean_amine n-Pentane mol frac', 'lean_amine H2S mol frac', 'lean_amine H2O mol frac', 'lean_amine MDEAmine mol frac',
    'lean_amine molar flow, kgmol/h', 'lean_amine H2S molar flow, kgmol/h',
    'rich_amine temperature, C', 'rich_amine mass flow, kg/h', 'rich_amine molecular weight', 'rich_amine H2S mol frac',
    'rich_amine molar flow, kgmol/h'
]]

y = sulphur[['rich_amine H2S molar flow, kgmol/h']]

# Собираем трансформер столбцов
ct = make_column_transformer(
    (MinMaxScaler(), [
    'feed_gas temperature, C', 'feed_gas mass flow, kg/h', 'feed_gas CO2 mol frac', 'feed_gas Methane mol frac',
    'feed_gas Ethane mol frac', 'feed_gas Propane mol frac', 'feed_gas i-Butane mol frac', 'feed_gas n-Butane mol frac',
    'feed_gas i-Pentane mol frac', 'feed_gas n-Pentane mol frac', 'feed_gas H2S mol frac', 'feed_gas H2O mol frac',
    'feed_gas MDEAmine mol frac', 'feed_gas molar flow, kgmol/h', 'feed_gas H2S molar flow, kgmol/h',
    'lean_amine temperature, C', 'lean_amine mass flow, kg/h', 'lean_amine CO2 mol frac',
    'lean_amine Methane mol frac', 'lean_amine Ethane mol frac', 'lean_amine Propane mol frac',
    'lean_amine i-Butane mol frac', 'lean_amine n-Butane mol frac', 'lean_amine i-Pentane mol frac',
    'lean_amine n-Pentane mol frac', 'lean_amine H2S mol frac', 'lean_amine H2O mol frac', 'lean_amine MDEAmine mol frac',
    'lean_amine molar flow, kgmol/h', 'lean_amine H2S molar flow, kgmol/h',
    'rich_amine temperature, C', 'rich_amine mass flow, kg/h', 'rich_amine molecular weight', 'rich_amine H2S mol frac',
    'rich_amine molar flow, kgmol/h'
    ])
)

# Создаем обучающую и тестовые выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Загружаем тренировочную выборку
ct.fit(X_train)

# Нормализуем данные
X_train_normal = ct.transform(X_train)
X_test_normal = ct.transform(X_test)

X_train_normal.shape, y_train.shape

((348704, 35), (348704, 1))

In [167]:
# Set random set
tf.random.set_seed(42)

# 1. Create a model
prod_h2s_molar_flow_model = tf.keras.Sequential([
    tf.keras.layers.Dense(35, activation='relu'),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(50, activation='relu'),
    tf.keras.layers.Dense(1)
])

# 2. Compile the model
prod_h2s_molar_flow_model.compile(
    loss='mae',
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    metrics=['mae']
)

# 3. Fit the model
prod_h2s_molar_flow_model.fit(X_train_normal, y_train, epochs=15)

Epoch 1/15
10897/10897 [==============================] - 24s 2ms/step - loss: 6.3000 - mae: 6.3000
Epoch 2/15
10897/10897 [==============================] - 25s 2ms/step - loss: 2.4034 - mae: 2.4034
Epoch 3/15
10897/10897 [==============================] - 25s 2ms/step - loss: 2.0796 - mae: 2.0796
Epoch 4/15
10897/10897 [==============================] - 23s 2ms/step - loss: 1.8258 - mae: 1.8258
Epoch 5/15
10897/10897 [==============================] - 25s 2ms/step - loss: 1.6625 - mae: 1.6625
Epoch 6/15
10897/10897 [==============================] - 24s 2ms/step - loss: 1.5271 - mae: 1.5271
Epoch 7/15
10897/10897 [==============================] - 23s 2ms/step - loss: 1.4344 - mae: 1.4344
Epoch 8/15
10897/10897 [==============================] - 25s 2ms/step - loss: 1.3312 - mae: 1.3312
Epoch 9/15
10897/10897 [==============================] - 23s 2ms/step - loss: 1.2572 - mae: 1.2572
Epoch 10/15
10897/10897 [==============================] - 24s 2ms/step - loss: 1.2158 - mae: 1.2158

In [168]:
y_pred = prod_h2s_molar_flow_model.predict(X_test_normal)

2725/2725 [==============================] - 5s 2ms/step


In [177]:
x = 7766
pred = y_pred[x][0].round(4)
pred

140.3744

In [178]:
test = round(y_test['rich_amine H2S molar flow, kgmol/h'].tolist()[x], 4)
test

141.2213

In [179]:
100 - (pred * 100) / test

0.5996935963437835

In [172]:
X_test['feed_gas H2S molar flow, kgmol/h']

219918    154.230875
371129    264.898626
366098     36.340909
455246     45.813944
195374    100.803536
             ...    
62275      29.947443
477786     62.308294
368128    166.398809
20897      30.933802
296449    360.571354
Name: feed_gas H2S molar flow, kgmol/h, Length: 87176, dtype: float64

In [173]:
y_test

,"rich_amine H2S molar flow, kgmol/h"
219918,48.578252
371129,269.886631
366098,38.992866
455246,47.719885
195374,102.594163
...,...
62275,31.219122
477786,63.642642
368128,171.993506
20897,35.507035


In [180]:
prod_h2s_molar_flow_model.save('./models/amine_treatment/amine_treatment_lean_amine_H2S_molar_flow.h5')